# What is IForest behavior according to the dataset dimension size?

## Test with 1 dimension dataset

In [1]:
import sys
sys.path.append('../../../../../')
from datasets import datasets as datat
from IForest_DODiMDS import iforest_D as IFD
from metrics import performances
from metrics import visualization
visu = visualization.visualization()

import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from imblearn.metrics import specificity_score
from imblearn.metrics import sensitivity_score
from imblearn.metrics import specificity_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve

import matplotlib.cm as cm
from mpl_toolkits import mplot3d

import eif

## Generals settings

In [2]:
max_samples = 256
n_trees = 100
threshold = 0.6

## Chargement du jeu de données

In [3]:
dts = datat.synthetic_datasets(path_top="../../../../../")
X_brut, y_transform, full_dataset = dts.OneD_V1_0()
X_brut.describe()

Columns number = 2


,0
count,106.000000
mean,0.148589
std,1.358918
min,-2.662713
25%,-0.665440
50%,-0.063532
75%,0.870154
max,4.114578


In [4]:
y_transform.describe()

,0
count,106.000000
mean,0.905660
std,0.426018
min,-1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [5]:
y_transform

,0
0,1
1,1
2,1
3,1
4,1
...,...
101,-1
102,-1
103,-1
104,-1


## Execution of Iforest to get the best performance

In [6]:
roc_aucs = []
specificities = []
for i in range(10):
    # Instanciation, fit and predict
    func_IFD = IFD.IsolationForest(sample_size = max_samples, n_trees=n_trees)
    func_IFD.fit(X_brut)
    IFD_y_pred_IF, IFD_scores = func_IFD.predict(X=X_brut, threshold=threshold)

    # Matrice de confusion
    ttn, tfp, tfn, ttp = confusion_matrix(y_transform, IFD_y_pred_IF).ravel()
    print(confusion_matrix(y_transform, IFD_y_pred_IF))
    #ROC AUC
    roc_aucs.append(roc_auc_score(y_transform, IFD_y_pred_IF))
    # Specificity
    specificities.append(specificity_score(y_transform, IFD_y_pred_IF))
print("ROC AUC")
print(roc_aucs)
print("Specificities")
print(specificities)
min_roc_auc = max(roc_aucs)
print("min_roc_auc = "+str(min_roc_auc))

[[ 2  3]
 [ 4 97]]
[[ 3  2]
 [ 4 97]]
[[ 4  1]
 [ 4 97]]
[[ 4  1]
 [ 4 97]]
[[ 2  3]
 [ 4 97]]
[[ 3  2]
 [ 4 97]]
[[ 5  0]
 [ 5 96]]
[[ 4  1]
 [ 4 97]]
[[ 2  3]
 [ 4 97]]
[[ 4  1]
 [ 4 97]]
ROC AUC
[0.6801980198019801, 0.7801980198019802, 0.8801980198019802, 0.8801980198019802, 0.6801980198019801, 0.7801980198019802, 0.9752475247524752, 0.8801980198019802, 0.6801980198019801, 0.8801980198019802]
Specificities
[0.4, 0.6, 0.8, 0.8, 0.4, 0.6, 1.0, 0.8, 0.4, 0.8]
min_roc_auc = 0.9752475247524752


In [7]:
(119/(119+1382))*100

7.928047968021319

### Execution Of EIF on dataset

EIF don't work for One dimensionnal dataset, try to see if it is cause of EIF implementation on Pypi or cause of EIF principle

In [8]:
X_brut_numpy = X_brut.to_numpy()

In [ ]:
F1  = eif.iForest(X_brut_numpy, ntrees=n_trees, sample_size=max_samples, ExtensionLevel=0)
S1 = F1.compute_paths(X_in=X_brut_numpy)
P1 = np.where(S1>=threshold,-1,1)

# Matrice de confusion
#ttn, tfp, tfn, ttp = confusion_matrix(y_transform_Shuttle, MS_y_pred_IF_Shuttle).ravel()
print(confusion_matrix(y_transform, P1))

EIF ne fonctionne pas sur les jeux de données à une seule dimension ===> Normal

In [ ]:
roc_auc_IF = []
max_samples_IF = []

for j in range(5, len(X_brut), 5):
    print("max_samples = "+str(j))
    max_samples = j
    # Instanciation, fit and predict
    func_IF = IFD.IsolationForest(sample_size = max_samples, n_trees=n_trees)
    func_IF.fit(X_brut)
    y_pred_IF, scores = func_IF.predict(X=X_brut, threshold=threshold)
    print(confusion_matrix(y_transform, y_pred_IF))

    auc_IF = roc_auc_score(y_transform, y_pred_IF)
    if auc_IF >= min_roc_auc:
        print("auc_IF = "+str(auc_IF))
        roc_auc_IF.append(auc_IF)
        max_samples_IF.append(j)

In [ ]:
#plt.plot(max_samples_IF, roc_auc_IF,  "b-", marker='.', markerfacecolor='blue', markersize=6, color='skyblue')
#plt.title("ROC AUC Min = "+str(min_roc_auc))
#plt.axis([0, 1, 0, 1])
#plt.xlabel('Max_samples')
#plt.ylabel('ROC AUC')
#plt.show()
visu.plot_2D(max_samples_IF, roc_auc_IF,'Max_samples', 'ROC AUC', "ROC AUC Min = "+str(min_roc_auc), link="b").show()

## Conclusion

- IForest arrive à détecter toutes les anomalies mais le taux de fausses alertes reste quand même élevé (environ 8%) pour les paramètres suivants : max_samples = 256, n_trees = 100, threshold = 0.5. Et ce taux de fausses alertes pourrait augmenter en fonction de la taille du jeu de données total.
Alors la question qui se pose est "Comment faire pour minimiser le taux de fausses alertes tout en conservant une bonne spécificité?"

- L'hypothèse selon laquelle la performance de IForest est fonction du nombre de données dans l'échantillon et par ricochet la taille maximale de l'arbre est vérifié car, on remarque que plus la taille de l'échantillon augmente, plus la AUC est meilleure même si on maintient n_trees = 100, threshold = 0.5 Ce qu'il y a lieu de faire est de vérifier:

    - si cette amélioration est dû au nombre de données anormales qui se retrouve dans l'échantillon ou si c'est dû uniquement à la taille de l'arbre
    - le rapport entre la taille de l'échantillon et le nombre d'arbres qu'il faut pour obtenir une bonne performance
    - le seuil à choisir dépend t-il d'un paramètre?
    - que faire quand les anomalies se ressemblent tellement qu'elles se retrouvent aussi au pied de l'arbre
    - si les noeuds feuilles de chaque arbre de la forêt ainsi créée contient des données anormales.